In [ ]:
from __future__ import annotations

INPUT_DIR = '../resources/'

def readfile(filename: str) -> [str]:
    with open(INPUT_DIR + filename, 'r') as file:
        return [line.strip() for line in file]

***

# Day One

In [ ]:
sorted_gnomes = []
current = 0
for line in readfile('day1.txt'):
    if line == '':
        sorted_gnomes.append(current)
        current = 0
    else:
        current += int(line)
sorted_gnomes.sort()

In [ ]:
# part 1
print(sorted_gnomes[-1])

In [ ]:
# part 2
print(sum(sorted_gnomes[-3:]))

***

# Day Two

In [ ]:
themMap = {'A':'X', 'B':'Y', 'C':'Z'}

def score(them, us):
    if them == us: # draw
        return 3 + (1 if us == 'X' else 2 if us == 'Y' else 3)
    if them == 'X':
        return 8 if us == 'Y' else 3
    if them == 'Y':
        return 9 if us == 'Z' else 1
    if them == 'Z':
        return 7 if us == 'X' else 2

rounds = readfile('day2.txt')

In [ ]:
# part one
total = 0
for r in rounds:
    total += score(themMap[r[0]], r[2])
total

In [ ]:
# part 2
def winLoseDraw(instr, themC):
    if instr == 'Z': # win
        return 'X' if themC == 'Z' else 'Y' if themC == 'X' else 'Z'
    if instr == 'X': # lose
        return 'X' if themC == 'Y' else 'Y' if themC == 'Z' else 'Z'
    return themC # draw

total2 = 0
for r in rounds:
    them = themMap[r[0]]
    total2 += score(them, winLoseDraw(r[2], them))

total2

***
# Day Three

In [ ]:
import string

bags = readfile('day3.txt')
priorities = {}
priority = 1
for c in string.ascii_lowercase + string.ascii_uppercase:
    priorities[c] = priority
    priority += 1

In [ ]:
# part 1
total = 0
for bag in bags:
    mid = int(len(bag)/2)
    (left,right) = ({c for c in bag[:mid]}, {c for c in bag[mid:]})
    i = left.intersection(right)
    # should be just 1 element
    for c in left.intersection(right):
        total += priorities[c]

total

In [ ]:
# part two
current = 0
total = 0
bagsN = len(bags)
while current < bagsN-2:
    group = [{c for c in bags[current+i]} for i in [0,1,2]]
    for c in group[0].intersection(group[1], group[2]):
        total += priorities[c]
    current += 3

total

***
# Day Four

In [ ]:
input = readfile('day4.txt')

def parse_line(line):
    xs = line.split(',')
    xs1, xs2 = xs[0].split('-'), xs[1].split('-')
    return (int(xs1[0]), int(xs1[1])), (int(xs2[0]), int(xs2[1]))

sections = list(map(parse_line, input))

In [ ]:
# part 1
count = 0
for first, second in sections:
    if (first[0] >= second[0] and first[1] <= second[1]) or (second[0] >= first[0] and second[1] <= first[1]):
        count += 1
count

In [ ]:
# part 2
count = 0
for first, second in sections:
    if (second[0] <= first[0] <= second[1]) or (second[0] <= first[0] <= second[1]) or (first[0] <= second[0] <= first[1]) or (first[0] <= second[1] <= first[1]):
        count += 1
count

***
# Day Five

In [ ]:
stacks_str = '''
 , , , , , ,R,J,W
 , , ,R,N, ,T,T,C
R, , ,P,G, ,J,P,T
Q, ,C,M,V, ,F,F,H
G,P,M,S,Z, ,Z,C,Q
P,C,P,Q,J,J,P,H,Z
C,T,H,T,H,P,G,L,V
F,W,B,L,P,D,L,N,G
'''

# parse stacks
stacks = {}
for row in stacks_str.split('\n'):
    stack = 1
    for col in row.split(','):
        if col != ' ' and col != '':
            if stack not in stacks:
                stacks[stack] = []
            stacks[stack].insert(0,col) # build stack 'backwards' so last elem is top
        stack += 1

def read(line):
    # move 2 from 2 to 8 -> (2,2,8)
    xs = line.split(' ')
    return int(xs[1]), int(xs[3]), int(xs[5])

# parse instructions
instructions = [read(line) for line in readfile('day5.txt')]

In [ ]:
import copy
# part one
stacks_c = copy.deepcopy(stacks)
for i in instructions:
    n, fromStack, toStack = i[0], stacks_c[i[1]], stacks_c[i[2]]
    for box in range (0,n):
        toStack.append(fromStack.pop())

print([stacks_c[i][-1] for i in range(1,10)])

In [ ]:
# part two
stacks_c2 = copy.deepcopy(stacks)
for i in instructions:
    n, fromStack, toStack = i[0], stacks_c2[i[1]], stacks_c2[i[2]]
    toStack.extend(fromStack[-n:])
    stacks_c2[i[1]] = fromStack[0:-n]

print([stacks_c2[i][-1] for i in range(1,10)])

***
# Day 6

In [ ]:
input = readfile('day6.txt')[0]

In [ ]:
def day6(N):
    n = N
    while n <= len(input):
        if len({x for x in input[n-N:n]}) == N:
            return n
        n += 1

In [ ]:
# part one
day6(4)

In [ ]:
# part two
day6(14)

***
# Day 7

In [ ]:
import re
input = readfile('day7.txt')

class SystemObject(object):
    name: str
    size: int = 0
    file: bool = True

    def __init__(self, name, size):
        self.name = name
        self.size = size

class Directory(SystemObject):
    parent: Directory
    contents: {str, SystemObject}

    def __init__(self, name, size, parent, contents):
        super().__init__(name, size)
        self.parent = parent
        self.contents = contents
        self.file = False

    def set_parent(self, dir: Directory):
        self.parent = dir

    def add_child(self, obj: SystemObject):
        self.size += obj.size
        self.contents[obj.name] = obj
        if obj.size > 0 and obj.file:
            self.__add_size__(obj.size)

    def __add_size__(self, size):
        # walk up the tree adding size of directory after adding a child object
        p = self.parent
        while p:
            p.size += size
            p = p.parent

# parse input
root = Directory('/', 0, None, {})
inputStack = input.copy()
inputStack.reverse()
current_dir = root

while inputStack:
    line = inputStack.pop()
    # change dir cmd
    cd = re.search('\$ cd (.*)', line)
    if cd:
        dir = cd.group(1)
        if dir == '..':
            current_dir = current_dir.parent
        elif dir != current_dir.name: # dir name
            current_dir = current_dir.contents[dir]
        continue

    # ls cmd
    if line == '$ ls': # read current dir's contents
            line = inputStack.pop() if inputStack else None
            while line and line[0] != '$':
                d = re.search('dir (.*)', line)
                if d: # read dir
                    d_name = d.group(1)
                    if d_name not in current_dir.contents:
                        current_dir.add_child(Directory(parent=current_dir, name=d_name, size=0, contents={}))
                else: # read file
                    xs = line.split(' ')
                    size, name = int(xs[0]), xs[1]
                    if name not in current_dir.contents:
                        current_dir.add_child(SystemObject(name, size))
                line = inputStack.pop() if inputStack else None

            if line and line[0] == '$':
                # push instruction back onto stack if cmd
                inputStack.append(line)


In [ ]:
# part one
total = 0
stack = [root]
dir = None
while stack:
    dir = stack.pop()
    if dir.size <= 100000:
        total += dir.size
    # search sub dirs
    for sub_dir in dir.contents:
        o = dir.contents[sub_dir]
        if not o.file:
            stack.append(o)
total

In [ ]:
# part two
min_space_required = 30000000 - (70000000 - root.size)
min_dir_so_far = root.size
stack = [root]
while stack:
    dir = stack.pop()
    if dir.size >= min_space_required:
        min_dir_so_far = min(min_dir_so_far, dir.size)
    # search sub dirs
    for sub_dir in dir.contents:
        o = dir.contents[sub_dir]
        if not o.file:
            stack.append(o)

min_dir_so_far

***
# Day 8

In [ ]:
# trees[row][col] = tree at (row,col)
trees = readfile('day8.txt')
trees = [[int(t) for t in line] for line in trees]
rows = len(trees)
cols = len(trees[0])

In [ ]:
# part one
def visible(row: int, col: int, grid) -> bool:
    # on border?
    if row == 0 or row == rows-1 or col == 0 or col == cols-1:
        return True
    h = grid[row][col]
    for dr,dc in [(1,0), (-1,0), (0,1), (0,-1)]:
        next_row, next_col = row + dr, col + dc
        while True:
            # reached the outside of the grid?
            if not (0 <= next_row < rows and 0 <= next_col < cols):
                return True
            h_next = grid[next_row][next_col]
            if h_next >= h: # higher or equal tree found on that path
                break
            # continue in that dir
            next_row += dr
            next_col += dc
    return False

count = 0
for row in range(0,rows):
    for col in range(0,cols):
        if visible(row,col,trees):
            count += 1

count

In [ ]:
# part two
def scenic_score(row: int, col: int, grid) -> int:
    h = grid[row][col]
    total = 1
    for dr,dc in [(1,0), (-1,0), (0,1), (0,-1)]:
        scenic = 0
        next_row, next_col = row + dr, col + dc
        while True:
            # reached the outside of the grid?
            if not (0 <= next_row < rows and 0 <= next_col < cols):
                break

            scenic += 1
            h_next = grid[next_row][next_col]
            if h_next >= h: # higher or equal tree found on that path
                break
            # continue in that dir
            next_row += dr
            next_col += dc
        if scenic > 0:
            total *= scenic
    return total

best_so_far = 0
for row in range(0,rows):
    for col in range(0,cols):
        s = scenic_score(row, col, trees)
        best_so_far = max(best_so_far, scenic_score(row,col,trees))

best_so_far

***
# Day 9

In [ ]:
def day9input(file):
    input = []
    for line in readfile(file):
        xs = line.split(' ')
        input.append((xs[0], int(xs[1])))
    return input

In [ ]:
# part one
head = 0, 0
tail = 0, 0
tail_ps = {tail}

def distant(h, t):
    return abs(h[0]-t[0]) >= 2 or abs(h[1]-t[1]) >= 2

def update(d):
    global head, tail
    dx, dy = 0, 1 # U
    if d == 'D':
        dx, dy = 0, -1
    if d == 'L':
        dx, dy = -1, 0
    if d == 'R':
        dx = 1, 0

    head = (head[0]+dx, head[1]+dy)
    if distant(head, tail):
        tail = (tail[0]-dx, tail[1]-dy)
        tail_ps.add(tail)


def day9_partone(input):
    global head, tail
    for move in input:
        d = move[0]
        n = move[1]
        for step in range(0,n):
            update(d)
    return len(tail_ps)

In [ ]:
# part one 
actual = day9input('day9.txt')
day9_partone(actual)

In [ ]:
# part two

# rope[0] = head, rope[9] = tail
rope = [(0,0) for x in range(0,10)]
tail_ps = {(0,0)}


for move in input:
    d = move[0]
    n = move[1]
    if d == 'U':

            # for step in range(0,n):
            #     head = (head[0],head[1]+1)


        # if d == 'D':
        #     for step in range(0,n):
        #         head = (head[0],head[1]-1)
        #         if distant(head,tail):
        #             tail = (head[0],head[1]+1)
        #             tail_ps.add(tail)
        #
        # if d == 'L':
        #     for step in range(0,n):
        #         head = (head[0]-1,head[1])
        #         if distant(head,tail):
        #             tail = (head[0]+1,head[1])
        #             tail_ps.add(tail)
        #
        # if d == 'R':
        #     for step in range(0,n):
        #         head = (head[0]+1,head[1])
        #         if distant(head,tail):
        #             tail = (head[0]-1,head[1])
        #             tail_ps.add(tail)




***
# Day 10

In [ ]:
# part one
cycles = [20, 60, 100, 140, 180, 220]
sum = 0
cycle = 1
r1 = 1
for i in readfile('day10.txt'):
    if cycle in cycles:
        sum += r1 * cycle
    if i == 'noop':
        cycle += 1
    else:
        n = int(i.split(' ')[1])
        cycle += 1
        if cycle in cycles:
            sum += r1 * cycle
        r1 += n
        cycle += 1

sum

In [ ]:
# part two
r1 = 1
col = 0
crt = ['','','','','','']

def update_crt_line(c):
    crt[c // 40] += '#' if r1-1 <= c % 40 <= r1+1 else ' '

for i in readfile('day10.txt'):
    update_crt_line(col)
    col += 1
    if i != 'noop':
        n = int(i.split(' ')[1])
        update_crt_line(col)
        col += 1
        r1 += n
crt

***
# Day 11

In [ ]:
class Monkey:
    id: int
    items = []
    operator = None
    test = None
    monkeyA = None
    monkeyB = None
    inspections = 0

    def __init__(self, id, items, operator, test):
        self.id = id
        self.items = items
        self.operator = operator
        self.test = test

    def set(self, a, b):
        self.monkeyA = a
        self.monkeyB = b

    def round(self):
        while self.items:
            item = self.items.pop(0)
            self.inspections += 1
            item1 = self.operator(item)
            item1 //= 3
            if self.test(item1):
                self.monkeyA.items.append(item1)
            else:
                self.monkeyB.items.append(item1)

    def __str__(self):
        return "{} - {} - {}".format(self.id, self.inspections, self.items)

example:
Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1

In [ ]:
# part one example
m0 = Monkey(0, [79,98], lambda x: x*19, lambda x: x % 23 == 0)
m1 = Monkey(1, [54,65,75,74], lambda x: x+6, lambda x: x % 19 == 0)
m2 = Monkey(2, [79, 60, 97], lambda x: x*x, lambda x: x % 13 == 0)
m3 = Monkey(3, [74], lambda x: x+3, lambda x: x % 17 == 0)

m0.monkeyA = m2
m0.monkeyB = m3
m1.monkeyA = m2
m1.monkeyB = m0
m2.monkeyA = m1
m2.monkeyB = m3
m3.monkeyA = m0
m3.monkeyB = m1

ms = [m0, m1, m2, m3]
for i in range(0,20):
    for m in ms:
        m.round()

for m in ms:
    print(m)


Monkey 0:
  Starting items: 54, 98, 50, 94, 69, 62, 53, 85
  Operation: new = old * 13
  Test: divisible by 3
    If true: throw to monkey 2
    If false: throw to monkey 1

Monkey 1:
  Starting items: 71, 55, 82
  Operation: new = old + 2
  Test: divisible by 13
    If true: throw to monkey 7
    If false: throw to monkey 2

Monkey 2:
  Starting items: 77, 73, 86, 72, 87
  Operation: new = old + 8
  Test: divisible by 19
    If true: throw to monkey 4
    If false: throw to monkey 7

Monkey 3:
  Starting items: 97, 91
  Operation: new = old + 1
  Test: divisible by 17
    If true: throw to monkey 6
    If false: throw to monkey 5

Monkey 4:
  Starting items: 78, 97, 51, 85, 66, 63, 62
  Operation: new = old * 17
  Test: divisible by 5
    If true: throw to monkey 6
    If false: throw to monkey 3

Monkey 5:
  Starting items: 88
  Operation: new = old + 3
  Test: divisible by 7
    If true: throw to monkey 1
    If false: throw to monkey 0

Monkey 6:
  Starting items: 87, 57, 63, 86, 87, 53
  Operation: new = old * old
  Test: divisible by 11
    If true: throw to monkey 5
    If false: throw to monkey 0

Monkey 7:
  Starting items: 73, 59, 82, 65
  Operation: new = old + 6
  Test: divisible by 2
    If true: throw to monkey 4
    If false: throw to monkey 3

In [ ]:
# part one
m0 = Monkey(0, [54, 98, 50, 94, 69, 62, 53, 85], lambda x: x*13, lambda x: x % 3 == 0)
m1 = Monkey(1, [71, 55, 82], lambda x: x+2, lambda x: x % 13 ==0)
m2 = Monkey(2, [77, 73, 86, 72, 87], lambda x: x+8, lambda x: x%19 ==0)
m3 = Monkey(3, [97, 91], lambda x:x+1, lambda x: x%17==0)
m4 = Monkey(4, [78, 97, 51, 85, 66, 63, 62], lambda x: x*17, lambda x: x%5==0)
m5 = Monkey(5, [88], lambda x: x+3, lambda x: x%7==0)
m6 = Monkey(6, [87, 57, 63, 86, 87, 53], lambda x: x*x, lambda x: x%11==0)
m7 = Monkey(7, [73, 59, 82, 65], lambda x: x+6, lambda x: x%2==0)

m0.set(m2, m1)
m1.set(m7, m2)
m2.set(m4, m7)
m3.set(m6, m5)
m4.set(m6, m3)
m5.set(m1, m0)
m6.set(m5, m0)
m7.set(m4, m3)

ms = [m0, m1, m2, m3, m4, m5, m6, m7]
for i in range(0,20):
    for m in ms:
        m.round()

for m in ms:
    print(m)


In [ ]:
333*337

In [ ]:
# part two
rounds = 10000


class Monkey2(Monkey):

    reduction = None

    def __init__(self, id, items, operator, test, reduction):
        super(Monkey2,self).__init__(id, items, operator, test)
        self.reduction = reduction

    def round(self):
        while self.items:
            item = self.items.pop(0)
            self.inspections += 1
            item1 = self.operator(item)
            item1 = self.reduction(item1)
            if self.test(item1):
                self.monkeyA.items.append(item1)
            else:
                self.monkeyB.items.append(item1)


m0 = Monkey2(0, [54, 98, 50, 94, 69, 62, 53, 85], lambda x: x*13, lambda x: x % 3 == 0)
m1 = Monkey2(1, [71, 55, 82], lambda x: x+2, lambda x: x % 13 ==0)
m2 = Monkey2(2, [77, 73, 86, 72, 87], lambda x: x+8, lambda x: x%19 ==0)
m3 = Monkey2(3, [97, 91], lambda x:x+1, lambda x: x%17==0)
m4 = Monkey2(4, [78, 97, 51, 85, 66, 63, 62], lambda x: x*17, lambda x: x%5==0)
m5 = Monkey2(5, [88], lambda x: x+3, lambda x: x%7==0)
m6 = Monkey2(6, [87, 57, 63, 86, 87, 53], lambda x: x*x, lambda x: x%11==0)
m7 = Monkey2(7, [73, 59, 82, 65], lambda x: x+6, lambda x: x%2==0)

m0.set(m2, m1)
m1.set(m7, m2)
m2.set(m4, m7)
m3.set(m6, m5)
m4.set(m6, m3)
m5.set(m1, m0)
m6.set(m5, m0)
m7.set(m4, m3)

ms = [m0, m1, m2, m3, m4, m5, m6, m7]
for i in range(0,rounds):
    for m in ms:
        m.round()

for m in ms:
    print(m)


In [ ]:
inspections = [m.inspections for m in ms]

In [ ]:
inspections.sort()
inspections[-1] * inspections[-2]